### Background info

Followed the 3-year reliability paper: https://doi.org/10.1016/j.neuroimage.2021.118516
1. "Using these source estimates, we then estimated the power of cortical activity in the canonical frequency bands: delta (2–4 Hz), theta (4–8 Hz), alpha (8–12 Hz), beta (15–30 Hz), low gamma (30–80 Hz), and high gamma (80–150 Hz). We used Welch's method for estimating power spectrum densities (PSD) per four-second epoch across each MEG recording, with 1-second sliding Hamming windows overlapping at 50%. We then standardized the PSD values at each frequency bin to the total power across the frequency spectrum. We then averaged PSD maps (ie. source estimates) across epochs for each participant to obtain one set of six PSD maps (one per frequency band) per participant per visit."
2. "We projected these maps onto the MNI ICBM152 brain template (Fonov et al., 2009) and applied a 3 mm FWHM smoothing kernel."
3. "Used single rater two-way mixed-effects model and absolute agreement definition, or ICC(A,1)... ICC estimates and their 95% confidence intervals were calculated using the Matlab Central file-exchange ICC.m function. This ICC calculation was applied at every vertex in the PSD maps to obtain spatially specific reliability estimates at each of the frequency bands. This resulted in an ICC map per frequency band."
4. "To further visualize the reliability of source power in each frequency band, regions of interest (Brainstorm “scouts”) in the frontal, parietal, temporal, and occipital lobes were applied to each participant's PSD map. The average power (relative to total spectral power) across each lobe was extracted for each participant and each visit. ICCs of these values were then calculated using the same ICC(A,1) model."
- "ICC .5 indicates poor reliability, values between .5 to .75 indicates moderate reliability, values between .75 and .9 indicates good reliability, and values greater than .9 indicate excellent reliability. Importantly, we evaluated the level of reliability based on the 95% confidence interval of the ICC estimate, not the estimate itself, since the interval reveals the chance that the true ICC value lands on any point between the bounds."


### Set up

In [26]:
# IMPORT PACKAGES
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import pickle as pkl
import mne
import copy as copy
from pathlib import Path
import csv
from mne.time_frequency import psd_array_welch

base_dir = "C:/meg/NVAR_ICC/derivatives"

# Things to loop through
# dictionary = { # OLD VERSION
#     "NVAR008": ["251016", "251017", "251023", "251113"], 
#     "NVAR010": ["251027", "251028", "251103", "251124"], 
#     "NVAR011": ["251030", "251031", "251106", "251127"]
# }
dictionary = { # NEW VERSION
    "NVAR008": ["Day1", "Day2", "Day3", "Day4"], 
    "NVAR010": ["Day1", "Day2", "Day3", "Day4"], 
    "NVAR011": ["Day1", "Day2", "Day3", "Day4"]
}
runs = ["rest1", "rest2"]
bands = ["alpha", "beta", "delta", "g_high", "g_low", "theta"]


### Morph and average

generic_taskfree_MEGIN.py Add this change: 
- stc_band_base = Path(os.path.basename(path2raw)).stem + f"_{inv_method}_{band}_stc"
- stc_band_path = os.path.join(stc_dir, stc_band_base)
- stc_band.save(stc_band_path, overwrite=True)


Here, create morph and apply
- The 3yr reliability paper applied a 3mm smoothing kernel, but can't do this in mne python! Caution that the smoothing param that is part of the morph function does not do what you think it does
- Save morphed_stc_per_band -> end up with 5 stcs per run per participant

Sources: 
- https://mne.tools/stable/auto_examples/inverse/morph_surface_stc.html

Then, compute average over rest1 and rest2!

In [ ]:
for subject in dictionary: 
    for session in dictionary[subject]:
        for band in bands: 
            for run in runs: 

                # Make file name
                stc_dir = os.path.join(base_dir, "sub_" + subject, session, "beamformer", "stc")
                stc_basename = os.path.join(stc_dir, "sub_" + subject + "_" + run + "_raw_tsss_beamformer_" + band + "_stc")
                print("Input: " + stc_basename)
                print("Output: " + stc_basename + "_morphed")

                # Read in file
                stc = mne.read_source_estimate(stc_basename)

                # Create morph
                morph = mne.compute_source_morph(
                    stc,
                    subject_from = 'sub_' + subject, 
                    subject_to = "fsaverage", # to fsaverage
                    subjects_dir = "C:/meg/NVAR_ICC/MRI/freesurfer/"
                    )
                # Apply morph
                stc_morphed = morph.apply(stc)
                stc_morphed.save(stc_basename + "_morphed", overwrite = True)

            # Make file name
            stc_dir = os.path.join(base_dir, "sub_" + subject, session, "beamformer", "stc")
            stc_morphed_rest1_name = os.path.join(stc_dir, "sub_" + subject + "_rest1_raw_tsss_beamformer_" + band + "_stc_morphed")
            stc_morphed_rest2_name = os.path.join(stc_dir, "sub_" + subject + "_rest2_raw_tsss_beamformer_" + band + "_stc_morphed")
            stc_average_name = os.path.join(stc_dir, "sub_" + subject + "_average_raw_tsss_beamformer_" + band + "_stc_morphed")
            
            # Compute average
            stc_morphed_rest1 = mne.read_source_estimate(stc_morphed_rest1_name)
            stc_morphed_rest2 = mne.read_source_estimate(stc_morphed_rest2_name)
            stc_average = (stc_morphed_rest1 + stc_morphed_rest2)/2
            stc_average.save(stc_average_name, overwrite = True)

In [ ]:
# CHECKING

rest1 = mne.read_source_estimate("C:/meg/NVAR_ICC/derivatives/sub_NVAR008/251113/beamformer/stc/sub_NVAR008_rest1_raw_tsss_beamformer_beta_stc_morphed")
rest2 = mne.read_source_estimate("C:/meg/NVAR_ICC/derivatives/sub_NVAR008/251113/beamformer/stc/sub_NVAR008_rest2_raw_tsss_beamformer_beta_stc_morphed")
average = mne.read_source_estimate("C:/meg/NVAR_ICC/derivatives/sub_NVAR008/251113/beamformer/stc/sub_NVAR008_average_raw_tsss_beamformer_beta_stc_morphed")
og = mne.read_source_estimate("C:/meg/NVAR_ICC/derivatives/sub_NVAR008/251113/beamformer/stc/sub_NVAR008_rest1_raw_tsss_beamformer_beta_stc_morphed")

rest1 = mne.read_source_estimate("C:/meg/NVAR_ICC/derivatives/sub_NVAR010/251027/beamformer/stc/sub_NVAR010_rest1_raw_tsss_beamformer_beta_stc_morphed")
rest2 = mne.read_source_estimate("C:/meg/NVAR_ICC/derivatives/sub_NVAR010/251027/beamformer/stc/sub_NVAR010_rest2_raw_tsss_beamformer_beta_stc_morphed")
average = mne.read_source_estimate("C:/meg/NVAR_ICC/derivatives/sub_NVAR010/251027/beamformer/stc/sub_NVAR010_average_raw_tsss_beamformer_beta_stc_morphed")
#og = mne.read_source_estimate("C:/meg/NVAR_ICC/derivatives/sub_NVAR011/251031/beamformer/stc/sub_NVAR011_rest1_raw_tsss_beamformer_alpha_stc")

brain = rest2.plot(
    subject = "fsaverage", 
    subjects_dir = "C:/meg/NVAR_ICC/MRI/freesurfer/", 
    initial_time = 0, 
    hemi = "both", 
    time_viewer = True
)

Using control points [0.0042183  0.00432386 0.0047425 ]
False


### ICC
What you have at this point is one morph-average stc plot per band per day (day 1, day 2, day 3, day 4)
For ICC calculation, followed this guide: https://rowannicholls.github.io/python/statistics/agreement/intraclass_correlation.html 

In [28]:
"""
Shape of stc_average: 
array([[0.0053271 ],
       [0.00452035],
       [0.005533  ],
       ...,
       [0.00313858],
       [0.003465  ],
       [0.00346861]], shape=(20484, 1), dtype=float32)
"""

# Loop through bands
# for band in bands: 
band = "alpha" # test

# Loop through vertices
# There are 20484 vertices
#for vertex in range(20484): 
    # For every vertex, build a dataframe for ICC
vertex = 2 # Test

df = pd.DataFrame(columns=["TARGET", "RATER", "RATING"])

# Add the value for this vertex for average-morph for each subject/session 
for subject in dictionary: 
    for session in dictionary[subject]: 
        
        stc_dir = os.path.join(base_dir, "sub_" + subject, session, "beamformer", "stc")
        stc_average_name = os.path.join(stc_dir, "sub_" + subject + "_average_raw_tsss_beamformer_" + band + "_stc_morphed")
        stc = mne.read_source_estimate(stc_average_name)
        power = stc.data[vertex]

        df.loc[len(df)] = [subject, session, power[0]]

# Compute ICC on this


In [29]:
df

,TARGET,RATER,RATING
0,NVAR008,Day1,0.007925
1,NVAR008,Day2,0.006022
2,NVAR008,Day3,0.004376
3,NVAR008,Day4,0.004313
4,NVAR010,Day1,0.010452
5,NVAR010,Day2,0.010554
6,NVAR010,Day3,0.010341
7,NVAR010,Day4,0.010703
8,NVAR011,Day1,0.006076
9,NVAR011,Day2,0.006303
